<a href="https://colab.research.google.com/github/Poojithavalli/Basic_python_programs/blob/main/crop_recommendation_using_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from google.colab import files
from IPython.display import display, HTML
import ipywidgets as widgets

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Upload dataset
uploaded = files.upload()

# Load dataset
file_name = next(iter(uploaded))
dataset = pd.read_csv(file_name)

# Preprocessing
X = dataset.drop(columns=['Crop'])
y = dataset['Crop']

# Convert categorical variables to dummy/indicator variables
X = pd.get_dummies(X, drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

# Train the GaussianNB model
model = GaussianNB()
model.fit(X_train, y_train)

# Create widgets
def create_widgets():
    district_widget = widgets.Dropdown(
        options=dataset['District_Name'].unique(),
        description='District:',
        style={'description_width': 'initial'}
    )

    soil_color_widget = widgets.Dropdown(
        description='Soil Color:',
        style={'description_width': 'initial'}
    )

    temperature_widget = widgets.FloatText(
        description='Temperature:',
        style={'description_width': 'initial'}
    )

    nitrogen_widget = widgets.Dropdown(
        description='Nitrogen:',
        style={'description_width': 'initial'}
    )

    phosphorus_widget = widgets.Dropdown(
        description='Phosphorus:',
        style={'description_width': 'initial'}
    )

    potassium_widget = widgets.Dropdown(
        description='Potassium:',
        style={'description_width': 'initial'}
    )

    ph_widget = widgets.Dropdown(
        description='pH:',
        style={'description_width': 'initial'}
    )

    rainfall_widget = widgets.Dropdown(
        description='Rainfall:',
        style={'description_width': 'initial'}
    )

    recommend_widget = widgets.Output()

    return (
        district_widget,
        soil_color_widget,
        temperature_widget,
        nitrogen_widget,
        phosphorus_widget,
        potassium_widget,
        ph_widget,
        rainfall_widget,
        recommend_widget
    )

# Update options for dependent widgets
def update_options(widget, values):
    widget.options = sorted(values)

# Update dependent widgets based on district and soil color
def update_dependent_widgets(change):
    district = district_widget.value
    soil_color = soil_color_widget.value

    if district and soil_color:
        filtered_data = dataset[
            (dataset['District_Name'] == district) &
            (dataset['Soil_color'] == soil_color)
        ]

        update_options(nitrogen_widget, filtered_data['Nitrogen'].unique())
        update_options(phosphorus_widget, filtered_data['Phosphorus'].unique())
        update_options(potassium_widget, filtered_data['Potassium'].unique())
        update_options(ph_widget, filtered_data['pH'].unique())
        update_options(rainfall_widget, filtered_data['Rainfall'].unique())
    else:
        for widget in [
            nitrogen_widget,
            phosphorus_widget,
            potassium_widget,
            ph_widget,
            rainfall_widget
        ]:
            widget.options = []

# Update soil color options based on district
def update_soil_color_options(change):
    district = change['new']

    if district:
        soil_colors = dataset[
            dataset['District_Name'] == district
        ]['Soil_color'].unique()

        update_options(soil_color_widget, soil_colors)
    else:
        soil_color_widget.options = []

# Function to predict crop
def predict_crop(b):
    input_data = pd.DataFrame({
        'District_Name': [district_widget.value],
        'Soil_color': [soil_color_widget.value],
        'Temperature': [temperature_widget.value],
        'Nitrogen': [nitrogen_widget.value],
        'Phosphorus': [phosphorus_widget.value],
        'Potassium': [potassium_widget.value],
        'pH': [ph_widget.value],
        'Rainfall': [rainfall_widget.value]
    })

    input_encoded = pd.get_dummies(input_data, drop_first=True)

    # Ensure all columns from training data are present
    for col in X.columns:
        if col not in input_encoded.columns:
            input_encoded[col] = 0

    # Reorder columns to match training data
    input_encoded = input_encoded[X.columns]

    prediction = model.predict(input_encoded)

    with recommend_widget:
        recommend_widget.clear_output()
        print(f"Recommended Crop: {prediction[0]}")

# Create widgets
(
    district_widget,
    soil_color_widget,
    temperature_widget,
    nitrogen_widget,
    phosphorus_widget,
    potassium_widget,
    ph_widget,
    rainfall_widget,
    recommend_widget
) = create_widgets()

# Observe changes
district_widget.observe(update_soil_color_options, names='value')
district_widget.observe(update_dependent_widgets, names='value')
soil_color_widget.observe(update_dependent_widgets, names='value')

# Create a button to predict crop
predict_button = widgets.Button(description="Predict Crop")
predict_button.on_click(predict_crop)

# Display widgets
display(widgets.VBox([
    district_widget,
    soil_color_widget,
    temperature_widget,
    nitrogen_widget,
    phosphorus_widget,
    potassium_widget,
    ph_widget,
    rainfall_widget,
    predict_button,
    recommend_widget
]))



Saving Crop and fertilizer dataset.csv to Crop and fertilizer dataset.csv
X_train shape: (3610, 311), y_train shape: (3610,)
